# 一、代码

In [ ]:
import pandas as pd
import jieba
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn
def chinese_word_cut(mytext):
    return ' '.join(jieba.cut(mytext))
def get_custom_stopwords(stop_words_file):
    with open(stop_words_file, encoding='utf-8') as f:
        stopwords = f.read( )
    stopwords_list = stopwords.split('\n')
    custom_stopwords_list = [i for i in stopwords_list]
    return custom_stopwords_list
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print('Topic #%d:' % topic_idx)
        print(' '.join([feature_names[i]
                      for i in topic.argsort()[:-n_top_words -1:-1]]))
    print()
df = pd.read_csv('data_20180601_20180605.csv', encoding='gb18030')
df.columns = ['title', 'author', 'comment'] 
X = df[['comment']]
X['cutted_comment'] = X.comment.apply(chinese_word_cut)
stop_words_file = 'mystopwords.txt'
stopwords = get_custom_stopwords(stop_words_file)
vect = CountVectorizer()
term_matrix = pd.DataFrame(vect.fit_transform(X.cutted_comment).toarray(), columns=vect.get_feature_names())
vect = CountVectorizer(stop_words=frozenset(stopwords)) #加上停用词去除功能
term_matrix = pd.DataFrame(vect.fit_transform(X.cutted_comment).toarray(), columns=vect.get_feature_names())
max_df = 0.8 # 在超过这一比例的文档中出现的关键词（过于平凡），去除掉。
min_df = 3 # 在地狱这一数量的文档中出现的关键词（过于独特），去除掉。
vect = CountVectorizer(max_df=max_df,
                      min_df=min_df,
                      token_pattern=u'(?u)\\b[^\\d\\W]\\w+\\b',
                      stop_words=frozenset(stopwords))
term_matrix = pd.DataFrame(vect.fit_transform(X.cutted_comment).toarray(), columns=vect.get_feature_names())
n_features = 1000
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                               max_features=n_features,
                               max_df=max_df,
                               min_df=min_df,
                               token_pattern=u'(?u)\\b[^\\d\\W]\\w+\\b',
                               stop_words=frozenset(stopwords))
tf = tf_vectorizer.fit_transform(X.cutted_comment)
n_topics = 5
lda = LatentDirichletAllocation(n_topics=n_topics,
                               max_iter=50,
                               learning_method='online',
                               learning_offset=50.,
                               random_state=0)
lda.fit(tf)
n_top_words = 20
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\wu\AppData\Local\Temp\jieba.cache
Loading model cost 0.916 seconds.
Prefix dict has been built succesfully.
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


# 二、结果：（默认5个主题）

In [ ]:
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

# 三、结果：（10个主题）
## 其中主题数可以自定义

In [ ]:
n_topics = 10    #此处的10可以自定义
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
lda.fit(tf)
print_top_words(lda, tf_feature_names, n_top_words)
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

# 四、结果：（20个主题）
## 其中主题数可以自定义

In [ ]:
n_topics = 20    #此处的20可以自定义
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
lda.fit(tf)
print_top_words(lda, tf_feature_names, n_top_words)
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)